In [2]:
!pip3 install simpletransformers

    100% |████████████████████████████████| 102kB 720kB/s a 0:00:01
    100% |████████████████████████████████| 204kB 1.8MB/s ta 0:00:011
    100% |████████████████████████████████| 10.1MB 186kB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/cd/80/5bb262050dd2f30f8819626b7c92339708fe2ed7bd5554c8193b4487b367/tqdm-4.42.1-py2.py3-none-any.whl
    100% |████████████████████████████████| 1.4MB 638kB/s ta 0:00:011
  Using cached https://files.pythonhosted.org/packages/dc/29/162476fd44203116e7980cfbd9352eef9db37c49445d1fec35509022f6aa/scipy-1.4.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/d1/48/e9fa9e252abcd1447eff6f9257636af31758a6e46fd5ce5d3c879f6907cb/scikit_learn-0.22.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/63/ae/057469010c579f6f6f37fc9b63b2d70af4ec20f2a7e2f3a1c4f364b4dde0/regex-2020.1.8-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/

  Using cached https://files.pythonhosted.org/packages/b9/63/df50cac98ea0d5b006c55a399c3bf1db9da7b5a24de7890bc9cfd5dd9e99/certifi-2019.11.28-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e8/74/6e4f91745020f967d09332bb2b8b9b10090957334692eb88ea4afe91b77f/urllib3-1.25.8-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/14/2c/cd551d81dbe15200be1cf41cd03869a46fe7226e7450af7a6545bfc474c9/idna-2.8-py2.py3-none-any.whl
    100% |████████████████████████████████| 378kB 1.0MB/s ta 0:00:01
    100% |████████████████████████████████| 593kB 845kB/s ta 0:00:01
    100% |████████████████████████████████| 71kB 750kB/s ta 0:00:011
    100% |████████████████████████████████| 71kB 1.4MB/s ta 0:00:011
  Using cached https://files.pythonhosted.org/packages/c7/48/a8252b6b3cd31774eab312b19d58a6ac55f296240c206617dcd38cd93bf8/s3transfer-0.3.2-py2.py3-none-any.whl
    100% |████████████████████████████████| 5.9MB 287kB/s ta 0:00:01
  Using cached https://

In [3]:
import os, re, string
import random

import numpy as np
import pandas as pd
import sklearn

import torch

from simpletransformers.classification import ClassificationModel
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [4]:
seed = 1337

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [6]:
train_data = pd.read_csv('../Desktop/train.csv')
train_data = train_data[['text', 'target']]

In [7]:
bert_uncased = ClassificationModel('bert', 'bert-large-uncased') 

# Print out all the default arguments for reference
bert_uncased.args

{'sliding_window': False,
 'tie_value': 1,
 'stride': 0.8,
 'regression': False,
 'output_dir': 'outputs/',
 'best_model_dir': 'outputs/best_model',
 'cache_dir': 'cache_dir/',
 'fp16': True,
 'fp16_opt_level': 'O1',
 'max_seq_length': 128,
 'train_batch_size': 8,
 'gradient_accumulation_steps': 1,
 'eval_batch_size': 8,
 'num_train_epochs': 1,
 'weight_decay': 0,
 'learning_rate': 4e-05,
 'adam_epsilon': 1e-08,
 'warmup_ratio': 0.06,
 'warmup_steps': 0,
 'max_grad_norm': 1.0,
 'do_lower_case': False,
 'logging_steps': 50,
 'save_steps': 2000,
 'no_cache': False,
 'save_model_every_epoch': True,
 'evaluate_during_training': False,
 'evaluate_during_training_steps': 2000,
 'evaluate_during_training_verbose': False,
 'use_cached_eval_features': True,
 'save_eval_checkpoints': True,
 'tensorboard_dir': None,
 'overwrite_output_dir': False,
 'reprocess_input_data': False,
 'process_count': 4,
 'n_gpu': 1,
 'use_multiprocessing': True,
 'silent': False,
 'wandb_project': None,
 'wandb_kwarg

In [8]:
# This is where we can tweak based on the default arguments above
custom_args = {'fp16': False, # not using mixed precision 
               'train_batch_size': 4, # default is 8
               'gradient_accumulation_steps': 2,
               'do_lower_case': True,
               'learning_rate': 1e-05, # using lower learning rate
               'overwrite_output_dir': True, # important for CV
               'num_train_epochs': 2} # default is 1

In [9]:
n=5
kf = KFold(n_splits=n, random_state=seed, shuffle=True)
results = []

for train_index, val_index in kf.split(train_data):
    train_df = train_data.iloc[train_index]
    val_df = train_data.iloc[val_index]
    
    model = ClassificationModel('bert', 'bert-base-uncased', args=custom_args) 
    model.train_model(train_df)
    result, model_outputs, wrong_predictions = model.eval_model(val_df, acc=sklearn.metrics.accuracy_score)
    print(result['acc'])
    results.append(result['acc'])


Converting to features started. Cache is not used.


/home/rubn/.local/lib/python3.6/site-packages/simpletransformers/classification/classification_model.py:244: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


Running loss: 1.284144


Running loss: 0.057286

Training of bert model complete. Saved to outputs/.
Converting to features started. Cache is not used.


/home/rubn/.local/lib/python3.6/site-packages/simpletransformers/classification/classification_model.py:574: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


...

Running loss: 0.422725

Training of bert model complete. Saved to outputs/.
Converting to features started. Cache is not used.



{'mcc': 0.639664101004443, 'tp': 503, 'tn': 752, 'fp': 126, 'fn': 141, 'acc': 0.8245729303547963, 'eval_loss': 0.49682678504608063}
0.8245729303547963
Features loaded from cache at cache_dir/cached_train_bert_128_2_6091


Running loss: 0.174044


Running loss: 0.067738

Training of bert model complete. Saved to outputs/.
Features loaded from cache at cache_dir/cached_dev_bert_128_2_1522



{'mcc': 0.6476577103604085, 'tp': 505, 'tn': 756, 'fp': 122, 'fn': 139, 'acc': 0.828515111695138, 'eval_loss': 0.4781928838313562}
0.828515111695138


In [10]:
for i, result in enumerate(results, 1):
    print(f"Fold-{i}: {result}")
    
print(f"{n}-fold CV accuracy result: Mean: {np.mean(results)} Standard deviation:{np.std(results)}")

Fold-1: 0.855548260013132
Fold-2: 0.8522652659225214
Fold-3: 0.8502954694681549
Fold-4: 0.8245729303547963
Fold-5: 0.828515111695138
5-fold CV accuracy result: Mean: 0.8422394074907485 Standard deviation:0.01298463890002993


In [11]:
model = ClassificationModel('bert', 'bert-base-uncased', args=custom_args) 
model.train_model(train_data)

Converting to features started. Cache is not used.


Running loss: 0.051356


Running loss: 0.102256

Training of bert model complete. Saved to outputs/.


In [12]:
test_data = pd.read_csv('../Desktop/test.csv')
predictions, raw_outputs = model.predict(test_data['text'])

sample_submission = pd.read_csv("../Desktop/sample_submission.csv")
sample_submission["target"] = predictions
sample_submission.to_csv("submission-k1.csv", index=False)

Converting to features started. Cache is not used.
